<a href="https://colab.research.google.com/github/utensil/llm-playground/blob/main/notebooks/axolotl/colab/monkeypatch_axolotl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone -b train https://github.com/utensil/axolotl.git

Cloning into 'axolotl'...
remote: Enumerating objects: 2614, done.
remote: Counting objects: 100% (890/890), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 2614 (delta 791), reused 721 (delta 685), pack-reused 1724
Receiving objects: 100% (2614/2614), 1.33 MiB | 9.54 MiB/s, done.
Resolving deltas: 100% (1627/1627), done.


In [2]:
%cd axolotl

/content/axolotl


In [3]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/axolotl
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-esok7o5q/transformers_6129b103593a488c99b68220b5a40d08
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-esok7o5q/transformers_6129b103593a488c99b68220b5a40d08
  Resolved https://github.com/huggingface/transformers.git to commit 8f093fb799246f7dd9104ff44728da0c53a9f67a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py

In [4]:
!pip install -U git+https://github.com/huggingface/peft.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-vajurb24
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-vajurb24
  Resolved https://github.com/huggingface/peft.git to commit 189a6b8e357ecda05ccde13999e4c35759596a67
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.4.0.dev0-py3-none-any.whl size=59308 sha256=414b6cadbcf4223f6eb472b8145ff607431f62b70e86d9b45b7ecced17ddd36c
  Stored in directory: /tmp/pip-ephem-wheel-cache-5vh9a2_r/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft


In [8]:
!accelerate config --config_file configs/accelerate/default_config.yaml default

2023-06-10 11:16:08.926713: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [10]:
!cp examples/falcon/config-7b-qlora.yml examples/falcon/config-1b-qlora.yml
!sed -i -e 's/falcon-7b/falcon-rw-1b/g' -e 's/wandb_project: falcon-qlora/wandb_project: /g' -e 's/bf16: true/bf16: false/g'  -e 's/tf32: true/tf32: false/g' examples/falcon/config-1b-qlora.yml

In [13]:
%%writefile scripts/ft.py
import finetune
from axolotl.utils.trainer import setup_trainer as setup_trainer_orig
import fire


def setup_trainer_ex(cfg, train_dataset, eval_dataset, model, tokenizer):
  print('setup_trainer_ex before')
  trainer = setup_trainer_orig(cfg, train_dataset, eval_dataset, model, tokenizer)
  print('setup_trainer_ex after')
  return trainer

finetune.setup_trainer = setup_trainer_ex

if __name__ == "__main__":
    fire.Fire(finetune.train)

Overwriting scripts/ft.py


In [ ]:
!accelerate launch scripts/ft.py examples/falcon/config-1b-qlora.yml

2023-06-10 11:28:36.216864: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/

# The rest are from earlier experiments that failed

In [ ]:
!sed -i -e 's/package_dir={""/package_dir={"axolotl"/g' setup.py

In [ ]:
!pip3 install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/axolotl
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-gr2t79ru/transformers_d68ea5c9193a4b6eb2346a90b507f559
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-gr2t79ru/transformers_d68ea5c9193a4b6eb2346a90b507f559
  Resolved https://github.com/huggingface/transformers.git to commit d0d1632958c7d543e07afc672a8501d704e5a65f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Attempting uninstall: axolotl
    Found existing installation: axolotl 0.1
    Uninstalling axolotl-0.1:
      Successfully uninstalled axolotl-0.1
  Running setup.py develop for axolotl


In [ ]:
import os
import sys
project_root = os.path.abspath(".")
project_root

'/content/axolotl'

In [ ]:
src_dir = os.path.join(project_root, "src")
sys.path.insert(0, src_dir)

In [ ]:
import axolotl

In [ ]:
import axolotl.utils

In [ ]:
from axolotl.utils.data import load_prepare_datasets

In [ ]:
import scripts.finetune as ft


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [ ]:
import importlib

In [ ]:
importlib.import_module('scripts.finetune')

<module 'scripts.finetune' from '/content/axolotl/scripts/finetune.py'>

# The rest are from earlier experiments to verify the feasibility of monkey patching

In [ ]:
%%writefile monk.py
class A:
  def inner(self):
    print ("inner() is being called")
  def caller(self):
    print ("caller() is being called")
    self.inner()

Overwriting monk.py


In [ ]:
import monk

In [ ]:
a = monk.A()

In [ ]:
a.caller()

caller() is being called
inner() is being called


In [ ]:
  def inner_patched(self):
    print ("inner_patched() is being called")

In [ ]:
monk.A.inner = inner_patched

In [ ]:
b = monk.A()

In [ ]:
b.caller()

caller() is being called
inner_patched() is being called
